In [1]:
#install RDKit package, check for more https://www.rdkit.org/docs/GettingStartedInPython.html
!pip install rdkit
#import useful packages
from rdkit import Chem              #basic molecular functionality
from rdkit.Chem import Draw         #creating images from molecules
from rdkit.Chem import MACCSkeys    #MACCS fingerprint
from rdkit.Chem import AllChem      #Coordinates can be generated
from rdkit import DataStructs       #culate molecular similarit
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
import numpy as np

In [2]:
#Input molecules and basic operations
# read the SMILES 
SMILES = "CN1C=NC2=C1C(=O)N(C(=O)N2C)C"
mol = Chem.MolFromSmiles(SMILES)
print("mol: ", mol)
atoms = mol.GetAtoms()
print("atoms: ", atoms)
print("\n")

# draw the structure
image = Draw.MolToImage(mol)
Draw.MolToFile(mol,"mol.png")  

# calculate the formula and number of atoms
formula = CalcMolFormula(mol)
num = mol.GetNumAtoms()
print("SMILES: ", SMILES)
print("formula: ", formula)
print("number of atoms: ", num)
print("\n")
# explore the molecule
for atom in atoms:
    atomic_num = atom.GetAtomicNum()
    total_Hs = atom.GetTotalNumHs()
    print("atomic_num: {:>2}, total_Hs: {:>2}".format(atomic_num, total_Hs))

for atom1 in range(mol.GetNumAtoms()):
    for atom2 in range(atom1+1, mol.GetNumAtoms()):
        bond = mol.GetBondBetweenAtoms(atom1, atom2)
        if bond is None:
            continue
        else:
            bond_order = bond.GetBondType()
            print("bond_order: {:>2}, is in ring? {:>2}".format(bond_order, bond.IsInRing()))

# canonicalization
def smiles_canonicalizer(smiles, isomericSmiles = True):
    mol = Chem.MolFromSmiles(smiles)
    canonical_smiles = Chem.MolToSmiles(mol, isomericSmiles=isomericSmiles)
    return canonical_smiles

output_smiles = smiles_canonicalizer(SMILES, isomericSmiles=False)
print("\n")
print("original smiles:  ", SMILES)
print("canonical smiles: ", output_smiles)

mol:  <rdkit.Chem.rdchem.Mol object at 0x000001E6FDA6A990>
atoms:  <rdkit.Chem.rdchem._ROAtomSeq object at 0x000001E6FDA6AB10>


SMILES:  CN1C=NC2=C1C(=O)N(C(=O)N2C)C
formula:  C8H10N4O2
number of atoms:  14


atomic_num:  6, total_Hs:  3
atomic_num:  7, total_Hs:  0
atomic_num:  6, total_Hs:  1
atomic_num:  7, total_Hs:  0
atomic_num:  6, total_Hs:  0
atomic_num:  6, total_Hs:  0
atomic_num:  6, total_Hs:  0
atomic_num:  8, total_Hs:  0
atomic_num:  7, total_Hs:  0
atomic_num:  6, total_Hs:  0
atomic_num:  8, total_Hs:  0
atomic_num:  7, total_Hs:  0
atomic_num:  6, total_Hs:  3
atomic_num:  6, total_Hs:  3
bond_order:  1, is in ring?  0
bond_order: 12, is in ring?  1
bond_order: 12, is in ring?  1
bond_order: 12, is in ring?  1
bond_order: 12, is in ring?  1
bond_order: 12, is in ring?  1
bond_order: 12, is in ring?  1
bond_order: 12, is in ring?  1
bond_order:  2, is in ring?  0
bond_order: 12, is in ring?  1
bond_order: 12, is in ring?  1
bond_order:  1, is in ring?  0
bond_order: 

In [3]:
#Molecular Fingerprint
#Use RDKit to generate MACCS, ECFP, and Coulomb matrix etc...
# MACCS 
maccs = MACCSkeys.GenMACCSKeys(mol)
maccs_bits = tuple(maccs.GetOnBits())
print("MACCS: ", maccs_bits)

# ECFP
ecfp = AllChem.GetMorganFingerprint(mol,2,invariants=[1]*mol.GetNumAtoms())

# Coulomb matrix
molh = Chem.AddHs(mol)
AllChem.EmbedMolecule(molh, AllChem.ETKDG())
coul = AllChem.CalcCoulombMat(molh)
coul = np.array(coul)
print("Coulomb Matrix ", coul)

MACCS:  (37, 38, 65, 75, 77, 79, 80, 83, 85, 89, 92, 93, 95, 96, 97, 98, 101, 105, 106, 110, 113, 117, 120, 121, 122, 125, 127, 136, 137, 141, 142, 143, 144, 148, 149, 150, 154, 156, 158, 159, 160, 161, 162, 163, 164, 165)
Coulomb Matrix  [[36.8581052  29.17984498 14.3745146  11.64260739  9.97829777 14.61917254
  11.61815815 16.50642529  9.47679755  7.04081441  7.54712175  8.76052486
   6.00773793  6.5972445   5.32404307  5.4509465   5.3454263   2.10646757
   0.99439478  0.88629579  0.91970929  0.95020958  1.02108008  1.15215172]
 [29.17984498 53.3587074  31.66691375 22.2937043  18.88905932 30.99992745
  16.72832504 19.07972626 13.29525025 10.37827126 10.5594257  14.03992499
   9.13409835  8.46827026  3.37195176  3.27541546  3.29931492  3.28001717
   1.51998751  1.29039892  1.36125393  1.22347351  1.29496604  1.39986542]
 [14.3745146  31.66691375 36.8581052  31.06784766 16.52573561 16.73749047
  10.22878589 11.48692444  9.4011707   8.09025874  8.50436398 11.96882729
   8.46539675  6.13

In [3]:
#Molecular Graph
# read the SMILES
SMILES = "C(C(C1C(=C(C(=O)O1)O)O)O)O"
mol = Chem.MolFromSmiles(SMILES)
# Adjacency matrix (Edge features)
adj = Chem.GetAdjacencyMatrix(mol, useBO=True)
print(f'adj: \n{ adj }')

# Node features
atoms = [a.GetSymbol() for a in mol.GetAtoms()]
length = len(atoms)  # number of atoms in the molecule
atom_dict = {}
for i in atoms:
  if i not in atom_dict:
    atom_dict[i] = len(atom_dict)
print(f'atom_dict: { atom_dict }')
n_atom = len(atom_dict)  # how many atom types

atoms = np.zeros((length, n_atom))
charge = np.zeros((length, 3))  # 3: formal charge [-1, 0, 1]
for atom in mol.GetAtoms():
  atoms[atom.GetIdx(), atom_dict[atom.GetSymbol()]] = 1
  charge[atom.GetIdx(), atom.GetFormalCharge() + 1] = 1

nodes = np.c_[atoms, charge]  # concatenate atom type and formal charge vectors
print(f'node features: \n{ nodes }')


adj: 
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 2. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 2. 0. 1. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 2. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
atom_dict: {'C': 0, 'O': 1}
node features: 
[[1. 0. 0. 1. 0.]
 [1. 0. 0. 1. 0.]
 [1. 0. 0. 1. 0.]
 [1. 0. 0. 1. 0.]
 [1. 0. 0. 1. 0.]
 [1. 0. 0. 1. 0.]
 [0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0.]]
